In [949]:
#Update your token
STUDENT_TOKEN = 'MAUDANDJOSPEH'

Server communication game and game_loop.

In [950]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print('\033[92mwinner\033[0m' if game.player == player else '\033[91mloser\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

Game specific code

In [951]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Domineering(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    actions = []
    board = self.get_board()
    player = self.player

    for row in range(len(board)):
      for col in range(len(board[row])):
        if board[row][col] == '.':
          if player == 'H' and col + 1 < len(board[row]) - 1 and board[row][col + 1] == '.':
            actions.append((row, col))
          elif player == 'V' and row + 1 < len(board) and board[row + 1][col] == '.':
            actions.append((row, col))
    return actions

  
  def print_game(self):
    for row in self.get_board():
      print(row)

  def other_player(self):
    if self.player == 'V': return 'H'
    if self.player == 'H': return 'V'
    


In [952]:
def actions(board, player):
  actions = []

  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == '.':
        if player == 'H' and col + 1 < len(board[row]) - 1 and board[row][col + 1] == '.':
          actions.append((row, col))
        elif player == 'V' and row + 1 < len(board) and board[row + 1][col] == '.':
          actions.append((row, col))
  return actions

def print_game(board):
  for row in board:
    print(row)
    
def heuristic(game, action, player):
  row, col = action
  temp_board = deepcopy(game.get_board())
  temp_board[row][col] = player
  opponent = game.other_player()
  board = game.get_board()
  
  def opponent_actions(temp_board, opponent, action):    
    possible_actions = actions(temp_board, opponent)
    print(f"Possible actions for opponent if move made in coordinate {action}", len(possible_actions))
    
    #also need to add player in either the right cell, or beneath depending on the player
    if player == 'V':
      temp_board[row + 1][col] = player
    elif player == 'H':
      temp_board[row][col + 1] = player
    
    print(f"Board with action in coordinate {action}")
    print_game(temp_board)
    
    return len(possible_actions)
  
  def player_actions(board, player, action):
    possible_actions = actions(board, player)
    print(f"Possible actions for player before move made in coordinate {action}", len(possible_actions))
    return len(possible_actions)
  
  opponent_moves = opponent_actions(temp_board, opponent, action)
  player_moves = player_actions(board, player, action)
  action_diff = opponent_moves - player_moves
  print("Action Diff: ", action_diff)
  
  return action_diff
    
    

def check_if_in(array1, array2):
  for element in array1:
    if element in array2:
      return True, list(set(array1) & set(array2))
  return False, None

def solver(game):
    preferred_orientation_H = [(0, 1), (0, 3), (5, 1), (5, 3)]
    preferred_orientation_V = [(1, 0), (3, 0), (1, 5), (3, 5)] 
    actions = game.actions()
    possible_moves = {}
    
    if game.player == 'H':
      bool, common_coordinates = check_if_in(preferred_orientation_H, actions)
      if bool:
        choice = random.choice(common_coordinates)
        print(choice)
        print("Checkpoint: took from preferred orientation")
        return choice
      
    elif game.player == 'V':
      bool, common_coordinates = check_if_in(preferred_orientation_V, actions)
      if bool:
        choice = random.choice(common_coordinates)
        print(choice)
        print("Checkpoint: took from preferred orientation")
        return choice
    
    for action in actions:
      number = heuristic(game, action, game.player)
      if number not in possible_moves.keys():
        possible_moves[number] = [action]
      else:
        possible_moves[number].append(action)
    
    #we want to have the maximum key value in possible_moves dictionary, and choose randomly from it
    #sort dictionary by keys, in descending order
    sorted_dict = dict(sorted(possible_moves.items(), key=lambda item: item[0], reverse=True))
    max_key = max(sorted_dict.keys())
    result_list = sorted_dict[max_key]
    
    print("Sorted Dictionary: ", sorted_dict)
    print("Max Key: ", max_key)
    print("Result List: ", result_list)
    choice = random.choice(result_list)
    print("Choice: ", choice)
    
    return choice
    

      

In [953]:
import random
def random_solver(game):
  actions = game.actions()
  print(game.actions())
  return random.choice(actions)

#for debugging
# game_state = [
#     ['H', 'H', '.', '.', 'H', 'H'],
#     ['H', 'H', '.', '.', '.', '.'],
#     ['V', 'V', '.', '.', '.', '.'],
#     ['V', 'V', '.', '.', '.', '.'],
#     ['V', '.', '.', '.', 'H', 'H'],
#     ['V', '.', '.', '.', '.', '.']
# ]

# game_turn = 'V'



In [954]:
game_loop(solver, Domineering, 'domineering', multi_player=False, id=None)

Creating new game...
Joining game with id: 3959
Playing as H
Waiting for the other player to join...
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', 'V', '.', '.', '.', '.']
['.', 'V', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
Making next move...
(0, 1)
Checkpoint: took from preferred orientation
b'Valid move'
['V', 'H', 'H', '.', '.', '.']
['V', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', 'V', '.', '.', '.', '.']
['.', 'V', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
Making next move...
(0, 3)
Checkpoint: took from preferred orientation
b'Valid move'
['V', 'H', 'H', 'H', 'H', '.']
['V', 'V', '.', '.', '.', '.']
['.', 'V', '.', '.', '.', '.']
['.', 'V', '.', '.', '.', '.']
['.', 'V', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
Making next move...
(5, 3)
Checkpoint: took from preferred orientation
b'Valid move'
['V', 'H', 'H', 'H', 'H', '.']
['V', 'V', '.', '.', '.', '.']
['.', 'V', '.', '.', '.'